In [20]:
import pandas as pd 
import numpy as np
import math
matchs = pd.read_csv("./data_scrapped/atp_matchs.csv")
matchs_stats = pd.read_csv("./data_scrapped/atp_matchs-stats.csv")
couk = pd.read_csv("./data_scrapped/co_uk_data.csv")

#"/en/players/jimmy-connors/c044/overview",
atp = matchs.join(matchs_stats, lsuffix='match_id',rsuffix='match_id')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
atp.head()

key=[]
for index, atpmatch in atp.iterrows():
    key.append(str(int(atpmatch.start_year)) + str(atpmatch.start_month) + atpmatch.winner_name.split(" ")[1] +  atpmatch.loser_name.split(" ")[1])


atp["key"] = key
atp.head()
#atp.iloc[0]
#print(all.loc[(all.winner_name == "Galo Blanco"),["winner_serve_rating","loser_name"]].describe())

#print(all.loc[(all.loser_name =="Galo Blanco"),["loser_serve_rating","loser_name"]].describe())

#+taill joueur
#+ranking joueurs
#match = matchs.iloc[0]
#match.winner_slug


ValueError: cannot convert float NaN to integer

In [84]:
import pandas as pd 
import numpy as np
from datetime import datetime

matchs = pd.read_csv("./data_scrapped/atp_matchs.csv")
matchs_stats = pd.read_csv("./data_scrapped/atp_matchs-stats.csv")
couk = pd.read_csv("./data_scrapped/co_uk_data.csv")
tournaments = pd.read_csv("./data_scrapped/atp_tournaments.csv")
players = pd.read_csv("./data_scrapped/atp_players.csv")

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# Sometimes atp matchs scrapped do not have accurate tournament date, 
# So we merge match en tournaments to take tournaments col we are more confident on
# matchs.loc[matchs.start_year.isna()] # = 550
# tournaments.loc[tournaments.year.isna()] # = 0
# dropping tournaments info from matchs
#matchs = matchs.drop(["tourney_order","tourney_name","tourney_slug","tourney_url_suffix","start_date","start_year","start_month","start_day","end_date","end_year","end_month", "end_day", "currency","prize_money","match_index"],axis=1)
matchs = matchs[["match_id", "tourney_year_id","tourney_round_name","round_order","match_order","match_score_tiebreaks",
                 "winner_name","winner_player_id","winner_slug","winner_games_won","winner_sets_won","winner_tiebreaks_won",
                 "loser_name","loser_player_id","loser_slug","loser_games_won","loser_sets_won","loser_tiebreaks_won"]]

tournaments = tournaments[["tourney_year_id","tourney_order","tourney_type","tourney_name",
                           "tourney_location","tourney_date","tourney_conditions","tourney_surface",
                           "currency","tourney_fin_commit"]]
tournaments["tourney_currency"] = tournaments["currency"]
tournaments = tournaments.drop(["currency"],axis=1)

atpmatchs = pd.merge(matchs, tournaments, how="left", left_on ="tourney_year_id", right_on = "tourney_year_id")

atpmatchs1player = pd.merge(atpmatchs, players, how="left", left_on ="winner_player_id", right_on = "player_id",  suffixes=('', '_winner'))
atpmatchs2player = pd.merge(atpmatchs1player, players, how="left", left_on ="loser_player_id", right_on = "player_id", suffixes=('_winner', '_loser'))

key=[]
for index, row in atpmatchs2player.iterrows():
    try:
        key.append(str(datetime.strptime(row.tourney_date,'%Y.%m.%d').year) + str(datetime.strptime(row.tourney_date,'%Y.%m.%d').month) + row.last_name_winner.split(" ")[0] +  row.last_name_loser.split(" ")[0])
    except:
        print(row)
atpmatchs2player["key"]=key


key=[]
couk = couk[["Date","Winner","Loser","WRank","LRank","B365W","B365L","PSW","PSL"]]
for index, row in couk.iterrows():
    key.append(str(datetime.strptime(row.Date,'%Y-%m-%d').year) + str(datetime.strptime(row.Date,'%Y-%m-%d').month) + row.Winner.split(" ")[0] +  row.Loser.split(" ")[0])
couk["key"]=key

atpmatchs_odds = pd.merge(atpmatchs2player, couk, how="left", left_on ="key", right_on = "key")


matchs_stats = matchs_stats[["match_id", "match_time","match_duration",
                             "winner_serve_rating","winner_aces","winner_double_faults","winner_first_serves_in",
                             "winner_first_serves_total","winner_first_serve_points_won","winner_first_serve_points_total",
                             "winner_second_serve_points_won","winner_second_serve_points_total","winner_break_points_saved","winner_break_points_serve_total",
                             "winner_service_games_played","winner_return_rating","winner_first_serve_return_won","winner_first_serve_return_total","winner_second_serve_return_won",
                             "winner_second_serve_return_total","winner_break_points_converted",
"winner_break_points_return_total",
"winner_return_games_played",
"winner_service_points_won",	"winner_service_points_total",
"winner_return_points_won",	"winner_return_points_total",
"winner_total_points_won",	"winner_total_points_total",
"loser_serve_rating",	"loser_aces",
"loser_double_faults",	"loser_first_serves_in",
"loser_first_serves_total",	"loser_first_serve_points_won",	"loser_first_serve_points_total",	"loser_second_serve_points_won",
"loser_second_serve_points_total",	"loser_break_points_saved",	"loser_break_points_serve_total",	"loser_service_games_played",	
"loser_return_rating",
"loser_first_serve_return_won",	"loser_first_serve_return_total",	"loser_second_serve_return_won",	
"loser_second_serve_return_total",	"loser_break_points_converted",
"loser_break_points_return_total",	"loser_return_games_played",	"loser_service_points_won",
"loser_service_points_total",	"loser_return_points_won",	"loser_return_points_total",
"loser_total_points_won",	"loser_total_points_total"]]

atpmatchs_stats_odds = pd.merge(atpmatchs_odds, matchs_stats, how="left", left_on ="match_id", right_on = "match_id")
atpmatchs_stats_odds.head()
#atpmatchs = pd.merge(matchs, tournaments, how="left", left_on ="tourney_year_id", right_on = "tourney_year_id")
#atpmatchs.head() # = 0

,match_id,tourney_year_id,tourney_round_name,round_order,match_order,match_score_tiebreaks,winner_name,winner_player_id,winner_slug,winner_games_won,winner_sets_won,winner_tiebreaks_won,loser_name,loser_player_id,loser_slug,loser_games_won,loser_sets_won,loser_tiebreaks_won,tourney_order,tourney_type,tourney_name,tourney_location,tourney_date,tourney_conditions,tourney_surface,tourney_fin_commit,tourney_currency,player_id_winner,first_name_winner,last_name_winner,country_code_winner,birthdate_winner,birth_year_winner,birth_month_winner,birth_day_winner,turned_pro_winner,weight_kg_winner,weight_lbs_winner,height_cm_winner,height_in_winner,birthplace_winner,player_id_loser,first_name_loser,last_name_loser,country_code_loser,birthdate_loser,birth_year_loser,birth_month_loser,birth_day_loser,turned_pro_loser,weight_kg_loser,weight_lbs_loser,height_cm_loser,height_in_loser,birthplace_loser,key,Date,Winner,Loser,WRank,LRank,B365W,B365L,PSW,PSL,match_time,match_duration,winner_serve_rating,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,winner_break_points_serve_total,winner_service_games_played,winner_return_rating,winner_first_serve_return_won,winner_first_serve_return_total,winner_second_serve_return_won,winner_second_serve_return_total,winner_break_points_converted,winner_break_points_return_total,winner_return_games_played,winner_service_points_won,winner_service_points_total,winner_return_points_won,winner_return_points_total,winner_total_points_won,winner_total_points_total,loser_serve_rating,loser_aces,loser_double_faults,loser_first_serves_in,loser_first_serves_total,loser_first_serve_points_won,loser_first_serve_points_total,loser_second_serve_points_won,loser_second_serve_points_total,loser_break_points_saved,loser_break_points_serve_total,loser_service_games_played,loser_return_rating,loser_first_serve_return_won,loser_first_serve_return_total,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_return_games_played,loser_service_points_won,loser_service_points_total,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,2000-451-2000-5-1-s424-s636,2000-451,Finals,1,1,36 75 30 (RET),Fabrice Santoro,s424,fabrice-santoro,13.0,2.0,0.0,Rainer Schuettler,s636,rainer-schuettler,11.0,1.0,0.0,1,NaN,Doha,Doha,2000.01.03,Outdoor,Hard,1000000,USD,s424,Fabrice,Santoro,FRA,1972.12.09,1972.0,12.0,9.0,1989.0,74.0,163.0,178.0,70.0,Tahiti,s636,Rainer,Schuettler,GER,1976.04.25,1976.0,4.0,25.0,1995.0,74.0,163.0,180.0,71.0,"Korbach, Germany",20001SantoroSchuettler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-451-2000-4-2-s424-k316,2000-451,Semi-Finals,2,1,75 64,Fabrice Santoro,s424,fabrice-santoro,13.0,2.0,0.0,Nicolas Kiefer,k316,nicolas-kiefer,9.0,0.0,0.0,1,NaN,Doha,Doha,2000.01.03,Outdoor,Hard,1000000,USD,s424,Fabrice,Santoro,FRA,1972.12.09,1972.0,12.0,9.0,1989.0,74.0,163.0,178.0,70.0,Tahiti,k316,Nicolas,Kiefer,GER,1977.07.05,1977.0,7.0,5.0,1995.0,80.0,177.0,183.0,72.0,"Holzminden, Germany",20001SantoroKiefer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-451-2000-4-1-s636-e121,2000-451,Semi-Finals,2,2,67(3) 64 64,Rainer Schuettler,s636,rainer-schuettler,18.0,2.0,0.0,Younes El Aynaoui,e121,younes-el-aynaoui,15.0,1.0,1.0,1,NaN,Doha,Doha,2000.01.03,Outdoor,Hard,1000000,USD,s636,Rainer,Schuettler,GER,1976.04.25,1976.0,4.0,25

In [101]:
import pandas as pd 
import numpy as np
from datetime import datetime


df = pd.read_csv("./data_scrapped/atp_matchs-stats.csv")

df['match_id'] = df['match_id'].str.lower()

#df.match_id = n_match_id
#df.head()
df.to_csv("./data_scrapped/atp_matchs-stats.csv",index=False)

,match_id,tourney_slug,match_stats_url_suffix,match_time,match_duration,winner_player_id,winner_serve_rating,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,winner_break_points_serve_total,winner_service_games_played,winner_return_rating,winner_first_serve_return_won,winner_first_serve_return_total,winner_second_serve_return_won,winner_second_serve_return_total,winner_break_points_converted,winner_break_points_return_total,winner_return_games_played,winner_service_points_won,winner_service_points_total,winner_return_points_won,winner_return_points_total,winner_total_points_won,winner_total_points_total,loser_player_id,loser_serve_rating,loser_aces,loser_double_faults,loser_first_serves_in,loser_first_serves_total,loser_first_serve_points_won,loser_first_serve_points_total,loser_second_serve_points_won,loser_second_serve_points_total,loser_break_points_saved,loser_break_points_serve_total,loser_service_games_played,loser_return_rating,loser_first_serve_return_won,loser_first_serve_return_total,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_return_games_played,loser_service_points_won,loser_service_points_total,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,2000-451-ms001-5-1-s424-s636,doha,/en/scores/2000/451/MS001/match-stats?isLive=False,01:58:00,118.0,s424,242.0,3.0,0.0,44.0,74.0,27.0,44.0,16.0,30.0,2.0,6.0,12.0,176.0,28.0,61.0,12.0,24.0,5.0,12.0,13.0,43.0,74.0,40.0,85.0,83.0,159.0,s636,237.0,1.0,1.0,61.0,85.0,33.0,61.0,12.0,24.0,7.0,12.0,13.0,186.0,17.0,44.0,14.0,30.0,4.0,6.0,12.0,45.0,85.0,31.0,74.0,76.0,159.0
1,2000-451-ms002-4-2-s424-k316,doha,/en/scores/2000/451/MS002/match-stats?isLive=False,01:39:00,99.0,s424,264.0,2.0,1.0,38.0,67.0,24.0,38.0,18.0,29.0,5.0,7.0,11.0,185.0,10.0,33.0,23.0,37.0,4.0,7.0,11.0,42.0,67.0,33.0,70.0,75.0,137.0,k316,215.0,1.0,4.0,33.0,70.0,23.0,33.0,14.0,37.0,3.0,7.0,11.0,122.0,14.0,38.0,11.0,29.0,2.0,7.0,11.0,37.0,70.0,25.0,67.0,62.0,137.0
2,2000-451-ms003-4-1-s636-e121,doha,/en/scores/2000/451/MS003/match-stats?isLive=False,02:35:00,155.0,s636,274.0,1.0,3.0,71.0,100.0,52.0,71.0,13.0,29.0,6.0,8.0,16.0,139.0,27.0,80.0,23.0,43.0,4.0,15.0,16.0,65.0,100.0,50.0,123.0,115.0,223.0,e121,256.0,6.0,3.0,80.0,123.0,53.0,80.0,20.0,43.0,11.0,15.0,16.0,119.0,19.0,71.0,16.0,29.0,2.0,8.0,16.0,73.0,123.0,35.0,100.0,108.0,223.0
3,2000-451-ms004-3-4-k316-s572,doha,/en/scores/2000/451/MS004/match-stats?isLive=False,01:07:00,67.0,k316,288.0,4.0,1.0,35.0,54.0,29.0,35.0,7.0,19.0,4.0,4.0,7.0,254.0,16.0,27.0,8.0,13.0,5.0,10.0,6.0,36.0,54.0,24.0,40.0,60.0,94.0,s572,162.0,1.0,2.0,27.0,40.0,11.0,27.0,5.0,13.0,5.0,10.0,6.0,80.0,6.0,35.0,12.0,19.0,0.0,4.0,7.0,16.0,40.0,18.0,54.0,34.0,94.0
4,2000-451-ms005-3-3-s424-b503,doha,/en/scores/2000/451/MS005/match-stats?isLive=False,01:18:00,78.0,s424,307.0,3.0,2.0,34.0,53.0,25.0,34.0,13.0,19.0,1.0,1.0,9.0,192.0,14.0,36.0,10.0,22.0,3.0,4.0,9.0,38.0,53.0,24.0,58.0,62.0,111.0,b503,245.0,1.0,0.0,36.0,58.0,22.0,36.0,12.0,22.0,1.0,4.0,9.0,58.0,9.0,34.0,6.0,19.0,0.0,1.0,9.0,34.0,58.0,15.0,53.0,49.0,111.0
